## Tracking multi faces

In [1]:
import cv2
import dlib
import face_recognition
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf


In [2]:
def load_data(path):
    face_encodings_in_room = []
    face_names_in_room = []
    known_face_encodings_array = np.load(path + "known_face_encoding.npy")
    known_face_names = np.load(path + "known_face_names.npy")
    
    # Convert nparray -> list to face encoding
    len_face = len(known_face_names)
    known_face_encodings_array = known_face_encodings_array.reshape(len_face, 128)
    known_face_encodings_list = []
    for i in range(len_face):
        known_face_encodings_list.append(known_face_encodings_array[i])
    
    return known_face_encodings_list, known_face_names

In [3]:
from threading import Thread
import cv2
import imutils
class WebcamVideoStream:
    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        self.stream.set(3, 800)
        self.stream.set(4, 600)
        (self.grabbed, self.frame) = self.stream.read()
        
        self.stopped = False
        
    def start(self):
        # Start the thread to read frames from the video stream
        Thread(target=self.update, args=()).start()
        return self
    
    def update(self):
        while True:
            if self.stopped:
                return
            
            (self.grabbed, self.frame) = self.stream.read()
            
    def read(self):
        return self.frame
    
    def stop(self):
        self.stopped = True

In [4]:
known_face_ecodings_list, known_face_names = load_data("../data/numpy/")

In [5]:
tracker = cv2.MultiTracker_create()

In [6]:
def computer_area(temp_box, list_box):
    dx = min(a.xmax,)
    

In [1]:
video_capture = WebcamVideoStream(src=0).start()
number_frame = 0
font = cv2.FONT_HERSHEY_SIMPLEX
tracker = cv2.MultiTracker_create()
init_once = False
name_boxes = []
boxes_known = []
name_known = []
while True:
    frame = video_capture.read()
#     rgb_frame = frame[:, :, ::-1]
    frame_process = frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    
    if (number_frame % 10 == 0):
        # Delete tracker and initi tracker
        
#         name_boxes = []
        if init_once == True:
            tracker.clear()
            tracker = cv2.MultiTracker_create()
#             # Create tracker with name known from before frame, and init name_boxes with people known
#             #
#             name_boxes_temp = [] # name of people 
#             boxes_known = [] # box of people known
            
#             for newbow, name in zip(boxes, name_boxes):
#                 if name != "unknown":
#                     ok = tracker.add(cv2.TrackerMIL_create(), frame, (newbow[0], newbow[1] , newbow[2], newbow[3]))
#                     name_boxes_temp.append(name)
#                     boxes_known.append(newbow)
#             print(name_boxes_temp)
#         # With people known, continue face recognition in next frame
#         # And only change name unknown if (area between unknown and predict > 90 %)
#             name_boxes = name_boxes_temp
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            if len(known_face_ecodings_list) > 0:
                distance = face_recognition.face_distance(known_face_ecodings_list, face_encoding)
                point = np.min(distance)
                index_point_min = np.argmin(distance)
#                 print(top, right, bottom, left)
                ok = tracker.add(cv2.TrackerMIL_create(), frame, (left, top, (right-left), (bottom-top)))

                if point > 0.4:
                    name = "Unknown"
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 127, 0), 3)
                    cv2.putText(frame, name, (left + 6, top - 6), font, 1.0, (0, 0, 255), 1)
                    name_boxes.append(name)
                elif point <= 0.4:
                    name = known_face_names[index_point_min]
                    cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 3)
                    cv2.putText(frame, name, (left + 6, top - 6), font, 1.0, (0, 0, 255), 1)
                    name_boxes.append(name)
    ok, boxes = tracker.update(frame)
    print(tracker)
    print(boxes)
    init_once = True
    for newbox, name in zip(boxes, name_boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, (0, 0, 255))
        cv2.putText(frame, name, p1, font, 1.0, (0, 0, 255), 1)
    cv2.imshow("frame", frame)
               
    number_frame += 1
               
    # Press Q on keyboard to stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

NameError: name 'WebcamVideoStream' is not defined